## Working With Dummy Data 

In [24]:
import numpy as np
import folium
import geopandas as gpd
import shapely
from shapely.geometry import Point
import folium 
import folium.plugins as HeatMap

In [25]:
ONTARIO_BOUNDS = {
    "min_lat": 41.7, "max_lat": 56.9,  # Latitude range
    "min_lon": -95.2, "max_lon": -74.3  # Longitude range
}

N_CANDIDATES = 100

# Generate random lat/lon points within Ontario bounds
lats = np.random.uniform(ONTARIO_BOUNDS["min_lat"], ONTARIO_BOUNDS["max_lat"], N_CANDIDATES)
lons = np.random.uniform(ONTARIO_BOUNDS["min_lon"], ONTARIO_BOUNDS["max_lon"], N_CANDIDATES)


In [26]:
geo_df = gpd.GeoDataFrame({"latitude": lats, "longitude": lons}, geometry=[Point(lon, lat) for lon, lat in zip(lons, lats)])
# geo_df.crs = "EPSG:4326"  # WGS84
# geo_df = geo_df.to_crs("EPSG:3857")  # Web Mercator
geo_df

,latitude,longitude,geometry
0,52.104825,-80.699220,POINT (-80.69922 52.10483)
1,49.063500,-75.067609,POINT (-75.06761 49.0635)
2,53.976565,-75.963523,POINT (-75.96352 53.97656)
3,43.861058,-77.313036,POINT (-77.31304 43.86106)
4,43.090154,-89.752878,POINT (-89.75288 43.09015)
...,...,...,...
95,43.339556,-77.831473,POINT (-77.83147 43.33956)
96,45.744803,-75.779155,POINT (-75.77916 45.7448)
97,41.713671,-83.853822,POINT (-83.85382 41.71367)
98,52.060425,-85.689024,POINT (-85.68902 52.06043)


In [27]:
highways = np.array([[-79.3832, 43.6532], [-75.6972, 45.4215]])  # Toronto & Ottawa (for simplicity)
power_grids = np.array([[-80.5, 43.7], [-78.5, 44.2]])  # Example power grid locations
population_centers = np.array([[-79.4, 43.7], [-75.7, 45.4], [-81.0, 42.9]])  # Toronto, Ottawa, London


In [28]:
def compute_distance_score(lat, lon, locations):
    return np.min(np.sqrt((locations[:, 0] - lon) ** 2 + (locations[:, 1] - lat) ** 2)) + 1  # Avoid division by zero

# Compute scores (inverse of distance)
scores = np.array([
    1 / compute_distance_score(lat, lon, highways) +  
    1 / compute_distance_score(lat, lon, power_grids) +  
    1 / compute_distance_score(lat, lon, population_centers)
    for lat, lon in zip(lats, lons)
])

# Select top candidates
top_indices = np.argsort(scores)[-20:]  # Select 20 best locations

# Initialize Folium Map centered on Ontario
map_center = [geo_df["latitude"].mean(), geo_df["longitude"].mean()]
m = folium.Map(location=map_center, zoom_start=6)

# Plot all Monte Carlo points
for lat, lon in zip(lats, lons):
    folium.CircleMarker(location=[lat, lon], radius=2, color="blue", fill=True, fill_opacity=0.4).add_to(m)

# Plot key infrastructure locations
# for lon, lat in highways:
#     folium.Marker([lat, lon], icon=folium.Icon(color="red", icon="road"), popup="Highway").add_to(m)

# for lon, lat in power_grids:
#     folium.Marker([lat, lon], icon=folium.Icon(color="green", icon="bolt"), popup="Power Grid").add_to(m)

# for lon, lat in population_centers:
#     folium.Marker([lat, lon], icon=folium.Icon(color="orange", icon="users"), popup="City Center").add_to(m)

# Plot top candidates
for idx in top_indices:
    folium.CircleMarker(location=[lats[idx], lons[idx]], radius=5, color="black", fill=True, fill_opacity=1, popup="Optimized Site").add_to(m)


In [29]:
m

In [ ]:
# Add heatmap
heat_data = [[lats[i], lons[i], scores[i]] for i in range(len(lats))]
HeatMap(heat_data, radius=15, blur=10).add_to(m)

# Save & display map
m.save("ev_charging_ontario.html")
m

In [ ]:
# data = (
#     np.random.normal(size=(100, 3)) * np.array([[1, 1, 1]]) + np.array([[48, 5, 1]])
# ).tolist()
# import folium
# from folium.plugins import HeatMap
# m = folium.Map([48.0, 5.0], zoom_start=6)
# HeatMap(data).add_to(m)
# m